**Decision Tree** - is a classifier expressed as a recursive partition of the instance space and consists of nodes that form a rooted tree. A directed tree has a root node that has no incoming edges whilst all the other nodes have exactly one incoming edge and the nodes are called leaves when they are the terminal or decision nodes.

Each leaf is assigned to one class representing themost appropiate target value or it may hold a probability vector indiciating the probability of the target attribute having a certain value. Decision Trees require very little data preparation and in particular, they do not require feature scaling or centering

*A node's value attribute tells one how many training instances of each class the node applies to and the gini attribute measures the impurity of a node, the purest node has gini = 0, meaning that all the training instances applied in the node belong to the same class*

$$ G_i = 1 -  \sum_{k= 1}^ {n} {p_i,k }^2 $$

where $p_ik$  is the ratio of class instances k amoung the training instances in the ith node

*A decision tree can also estimate the probability that an instance belongs to a particular class k : first it traverses the tree to find the leaf node for this instance and then it returns the ratio of training instances of class k in this node*

## Classification and Regression Tree Algorithm (CART)

Scikit Leanr uses the CART algorithm to grow decision trees by first splitting the training set into two subsets using a single feature  k and a threshold $t_k$  by searching for a pair $(k, t_k)$ that produces the purest subsets weighted by their size. The cost function tries to minimize the equation given below:

$$J(k, t_k) =  \frac{m_{(left)}}{m}G_ {(left)} +  \frac{m_right}{m}G_{right}$$

where $ \begin{cases} G_ {(left)}/right, &  \text {measures the impurity of the left/right subset} \\ m, & \text{is the number of instances in the left/right subset}  \end {cases}$

**Entropy** - *is an impurity measure that is zero when it contains instances of only one class*

$$H_i = - \sum_{k= 1}^{n} p_i,k log(p_i,k)$$

Gini imputiry is slightly faster to compute and isolates the most frequent class on its own branch in the tree whilst entropy tends to produce slightly more balanced trees.

For Regression, the CART algorithm tries to split the training set in a way that minimizes the cost function instead of the impurity as in classification. The predicted value for each leaf node is the simply the average target value of all the training instances associted with the leaf node.

*The cost function of a Regression Tree is given as*

$$J(k,t_k) = \frac{m_{left}}{m} MSE_{left} + \frac{m_{right}}{m} MSE_{right}$$ 

where $ \begin{cases} MSE_ {node} =\sum_{i\in node } (\hat{y}_{node}-y^{i})^2 \\ \hat{y}_{node} = \frac{1}{m_{node}}\sum_{i \in node} y^{(i)} \end {cases}$

Decision trees work best with orthogonal decision boundaries and this makes them sensitive to training set rotation and any other changes to the training set.

In [1]:
import warnings; warnings.simplefilter("ignore")
#importing important libraries
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.formula.api as  sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import csv
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor

In [2]:
# datafile upload and data preparation
df = pd.read_csv("diffreport2.csv", sep= ",")

d1 = df.drop("name", axis = 1)
d2 = d1.drop("isotopes", axis = 1)
d3 = d2.drop("adduct", axis = 1)
d4 = d3.drop("tstat", axis = 1)
d5 = d4.drop("pvalue", axis = 1)
d6 = d5.drop("fold", axis = 1)
d7 = d6.drop(d6.columns[0], axis = 1)
d8 = d7.drop("npeaks", axis = 1)
d9 = d8.drop("Eta6", axis = 1)
d10 = d9.drop("Eta8", axis = 1)
columns = ['Eta6_0', 'Eta6_2', 'Eta6_3', 'Eta8.1', 'Eta82', 'Eta83', 'ID']
df1 = pd.DataFrame(d10, columns = columns)

In [3]:
# creation of train and testing sets

def get_train_test(df, y_col, x_cols, ratio):

    mask = np.random.rand(len(df)) > ratio
    df_train = df[mask]
    df_test = df[~mask]
       
    Y_train = df_train[y_col].values
    Y_test = df_test[y_col].values
    X_train = df_train[x_cols].values
    X_test = df_test[x_cols].values
    return df_train, df_test, X_train, Y_train, X_test, Y_test
 
y_col = 'ID'
x_cols = list(df1.columns.values)
x_cols.remove(y_col)
 
train_test_ratio = 0.7
df_train, df_test, X_train, Y_train, X_test, Y_test = get_train_test(df1, y_col, x_cols, train_test_ratio)

### Classification Tree

In [4]:
tree_clf = DecisionTreeClassifier(max_depth = 2)
tree_clf.fit(X_train,Y_train)


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [11]:
#drawing a decision tree
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus

#drawing a decision tree
from sklearn.tree import export_graphviz
export_graphviz(
               tree_clf,
                out_file = "classification_tree.dot",
                max_depth = None,
                feature_names = None,
                rounded = True,
                filled = True
)

In [ ]:
import pydotplus

(graph,) = pydotplus.graph_from_dot_file('classification_tree.dot')
graph.write_png('classification_tree.png')

### Regression Trees

In [7]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(max_depth = 2)
tree_reg.fit(X_train,Y_train)

DecisionTreeRegressor(criterion='mse', max_depth=2, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

## Ensemble Learning and Random Forests

**Ensemble Learning** - is a machine learning paradigm whereby multiple learners (classifiers/regressors) are trained to solve the same problem by constructing a set of hypotheses and combining them.New data points are classified of regressed by taking a weighted vote of their predictions and these include Bayesian Averaging, error correcting output coding, Bagging and boosting.

**Hard Voting Classifier**- is a majority voting classifier that aggregates the predictions of each classifier and predicts the class that gets the most votes.Even if each classifier is a weak leaner( meaning it slightly outperforms random guessing), the ensemble  can still be a strong learner provided there are a sufficient number of weak learners and they are  sufficiently diverse.

Ensemble methods work best when the predictors are as independent from one another as possible.One way to get diverse classifiers is to train them using very different algorithms as this increases the chance that they will make  very different types of errors, improving the ensemble's accuracy.


In [16]:
# ensemble of random forest regressor, Support Vector Machine and Logistic Regression

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVR
from sklearn import preprocessing
from sklearn import utils

log_clf = LogisticRegression()
rnd_reg = RandomForestRegressor()
svm_reg = LinearSVR()

voting_clf = VotingClassifier(
              estimators = [('lr', log_clf), ('rf', rnd_reg), ('svc', svm_reg)],
              voting = 'soft')


voting_clf.fit(X_train, Y_train)

VotingClassifier(estimators=[('lr', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)), ('rf', RandomF...ing=1.0, loss='epsilon_insensitive', max_iter=1000,
     random_state=None, tol=0.0001, verbose=0))],
         flatten_transform=None, n_jobs=1, voting='soft', weights=None)

In [ ]:
#comparing each classifier's accuracy
from sklearn import preprocessing
from sklearn import utils
lab_enc= preprocessing.LabelEncoder()
encoded  = lab_enc.fit_transform(Y_train)

from sklearn.metrics import accuracy_score
for clf in ( rnd_reg, voting_clf):
    clf.fit(X_train, Y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(Y_test, y_pred))

### Bagging and Pasting 

**Bagging ** - is short for bootstrap aggregating and it trains a number of base learners from a different bootstrap sample by calling a base learning algorithm. The bootstrap sample is obtained by subsampling the training data set with replacement, where  the size of a sample is as the same as that of the training data set. After obtaining the base learners, Bagging combines them by majority voting and the most voted class is predicted.When sampling is performed without replacement, it is called ** "Pasting"**.

*Bagging and pasting allow training instances to be sampled several times across multiple predictors but only bagging allows training instances to be sampled several times from the same predictor. Once all the predictors are trained, the ensemble can make a prediction for a new instance by simply aggregating the predictions of all the predictors and the aggregation function is typically the statistical mode (most frequent prediction) for classification or the average for regression*

In [23]:
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor

bag_reg = BaggingRegressor(
           DecisionTreeRegressor(), n_estimators = 500,
          max_samples = 50, bootstrap = True, n_jobs = -1)  # bootstrap = False leads to pasting 
bag_reg.fit(X_train, Y_train)    # n_jobs denotes the number of processors dedicated to the the process
y_pred = bag_reg.predict(X_test)

Each bootstramp sample leaves out about 37% of the instances that are not sampled that are called OOB ** Out of Bag**, which are never the same for all the predictors. A predictor can be evaluated on the oob instances without the need for a seperate validation set or cross validation by averaging out the oob evaluations of each predictor.

In [24]:
# Out of Bag evaluation

bag_reg = BaggingRegressor(
          DecisionTreeRegressor(), n_estimators = 500,
          bootstrap = True, n_jobs = -1, oob_score = True)
bag_reg.fit(X_train, Y_train)
bag_reg.oob_score_

0.58563450352328483

In [ ]:
from sklearn.metrics import accuracy_score
y_pred = bag_reg.predict(X_test)
accuracy_score(Y_test, y_pred)

Random Patches - involves sampling both training instances and features. Keeping all training instances but sampling features  is called Random Subspaces method.

## Random Forest(Random Decision Forests)

Random forest is an ensemble learning method of a multitude of Decision Trees and outputs a class that is a mode of the classes(classification) and mean prediction (regression). Random Forests are usually trained via the bagging method. The hyperparameters of the tree are used to control how the trees are grown whilst the hyperparameter of teh BaggingClassifier are used to control the ensemble itself.

In [27]:
from sklearn.ensemble import RandomForestRegressor

rnd_reg = RandomForestRegressor(n_estimators = 500, max_leaf_nodes = 16, n_jobs = -1)
rnd_reg.fit(X_train,Y_train)

y_pred_rf = rnd_reg.predict(X_test)

In [28]:
# an equivalent bagging regressor 

bag_reg = BaggingRegressor(
          DecisionTreeRegressor(splitter = "random", max_leaf_nodes = 16),
          n_estimators = 500, max_samples = 1.0, bootstrap = True, n_jobs = -1)

Randomization methods, such as bagging, introduce randomization into the learning algorithm and and exploit at each run, a different randomized version of the original learning sample so as to produce an ensemble of more or less strongly diversified models.
**Extremely Randomized Trees** - *are structures independent of the target variabl evalues of the learning sample.The extra tree algorithm builds an ensemble of unpruned decision or regression trees according to the classical top down procedure with differences arising the fact that it splits nodes by choosing cut points fully at random and uses the whole learning sample(rather than a bootstrap replica) to grow the trees. Trees are made even more random  by using random thresholds for each feature rather than searching for the best possible thresholds*

**Feature importance**-*considers how much tree nodes that use a specific feature  reduce impurity on average or more precisely, look at the weighted average, where each node's weight is equal to the nuber of training samples that are associated with it*

In [29]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators = 500, n_jobs = -1)


**Boosting** - is an ensemble method that combines several weak learners into a strong learner by training predictors sequantially,each trying to correct its predecessor. **AdaBoost** assigns equal weights to all the training examples and then uses the training examples to test the base learner whilst increasing the weights of the incorrectly classified examples. The process is increased a number of times, each which is called a round up until the final learner is derived by weighted majority voting of the T ase learners where the weights of the learners are determined during the training process. Once all the predictors are trained, the ensemble makes predictions very much like bagging or pasting, except that predictors have different weights depending on their overall accuracy on the weighted training set.

Weighted error of the jth predictor is 

$$ r_j = \frac{\sum\limits_{i= 1}^ {m} w^{(i)}}{\sum\limits_{i= 1}^ {m} w^{(i)}} $$

predictor weight 

$$ \alpha_j = \eta log \frac{1-r_j}{r_j}$$

Scikit uses a multiclass version of AdaBoost called SAMME which stands for Stagewise Additive Modelling using a Multiclass Exponential Loss Function.The AdaBoost Classifier is based on *200 Decision Stumps* , a Decision Stump is a Decision Tree with a $max_depth = 1$ or a tree composed of a single decision node plus two leaf nodes. 

In [30]:
from sklearn.ensemble import AdaBoostRegressor

ada_reg = AdaBoostRegressor(
          DecisionTreeRegressor(max_depth = 1), n_estimators = 200,
           learning_rate = 0.5)
ada_reg.fit(X_train, Y_train)

AdaBoostRegressor(base_estimator=DecisionTreeRegressor(criterion='mse', max_depth=1, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best'),
         learning_rate=0.5, loss='linear', n_estimators=200,
         random_state=None)

## Gradient Boosting

The learning procedure consecutively fits new models to provide more accurate estimate of the response variable. The principle aidea is based on constructing new base learners to be maximally correlated with the negative gradient of the loss function associated with the whole ensemble.The loss function applied can be arbitrary and if the error function is the classic squared error loss,the learning procedure would result in a consecutive error fitting, therefore the loss function is up to the user.Instead of tweaking the instance weights at every iteration like Adaboost does, the method tries to fit the new predictor to the residual errors made by the previous predictor.

**Stochastic Gradient Boosting** - trades a higher biase for a lower variance and is a method that constructs additive regression models by sequentially fitting a simple parameterized function(base learner) to current pseudo-residuals by least squares at each iteration.

**Stacking** - in a typical implementation , a number of first level individual leaners are generated from training data set by employing different learning algorithms. The individual learners are then combined by a second level learner(meta learner).

**Generalization** - the generalization of an ensemble is usually much stronger than that of a single learner because the training data might notprovide sufficient information for choosing a single best learner or the search processes of the learning algoritms might be imperfect and lastly, the hypothesis space being searched might not contain the true target function, while ensembles van give some good approximation.

**Bias-variance decomposition**- is often used in studying the performance of ensemble methods. Bagging significantly reduces the variance and is best applied to decision trees or neural networks whilst boosting reduces the biase in addition to the variance and therefore is ideal for weak learners such as decision stumps.


In [34]:
from sklearn.tree import DecisionTreeRegressor

tree_reg1 = DecisionTreeRegressor(max_depth = 2)
tree_reg1.fit(X_train, Y_train)

# now train a second DecisionTreeRegressor on the residual errors made by the first predictor

y2 = Y_train - tree_reg1.predict(X_train)
tree_reg2 = DecisionTreeRegressor(max_depth = 2)
tree_reg2.fit(X_train, y2)

# now train a third regressor  on the residual errors made by the second predictor

y3 = y2-tree_reg.predict(X_train)
tree_reg3 = DecisionTreeRegressor(max_depth = 2)
tree_reg3.fit(X_train,y3)

#the ensemble of three trees can now be used to make new predictions

y_pred = sum(tree.predict(X_test) for tree in (tree_reg1, tree_reg2, tree_reg3))

In [ ]:
#Implementation of early stopping algorithm as a regularization method

import numpy as np
from sklearn.model_selection  import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor

X_train, X_val, Y_train, y_val = train_test_split(X_train,Y_train)

gbrt = GradientBoostingRegressor(max_depth = 2, n_estimators = 120)
gbrt.fit(X_train, Y_train)

errors = [mean_squared_error(y_val, y_pred)
         for y_pred in gbrt.staged_predict(X_val)]
bst_n_estimators = np.argmin(errors)

gbrt_best = GradientBoostingRegressor(max_depth = 2, n_estimators= bst_n_estimators)
gbrt_best.fit(X_train, Y_train)

In [ ]:
#the following code stops training when the validation error does not improve for five iterations in a row

gbrt = GradientBoostingRegressor(max_depth = 2, warm_start = True)

min_val_error = float("inf")
error_going_up = 0
for n_estimators in range(1, 120):
    gbrt.n_estimators = n_estimators
    gbrt.fit(X_train, y_train)
    y_pred = gbrt.predict(X_val)
    val_error = mean_squared_error(y_val, y_pred)
    if val_error < min_val_error:
        min_val_error = val_error
        error_going_up = 0
    else:
        error_going_up += 1
        if error_going_up ==5:
            break #early stopping